In [1]:
# =====================================================
# Import Library
# =====================================================
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
from skimage.feature import graycomatrix, graycoprops
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, Reshape, Lambda
from tensorflow.keras import backend as K

In [2]:
# =====================================================
# Load dataset dari folder
# =====================================================
def load_dataset(dataset_path):
    images_gray = []
    labels = []
    classes = os.listdir(dataset_path)
    for cls in classes:
        cls_path = os.path.join(dataset_path, cls)
        if not os.path.isdir(cls_path):
            continue
        for img_name in os.listdir(cls_path):
            img_path = os.path.join(cls_path, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                _, thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
                clean = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, np.ones((3,3),np.uint8))
                images_gray.append(clean)
                labels.append(cls)
    return images_gray, labels

In [3]:
# =====================================================
# Fitur fisik dan GLCM
# =====================================================
def categorize_length_mm(length_mm):
    if 6.5 <= length_mm < 7.0:
        return "pendek"
    elif 7.0 <= length_mm <= 8.5:
        return "panjang"
    else:
        return "lainnya"

def categorize_area_cm2(area_cm2):
    if area_cm2 < 1.0:
        return "kecil"
    elif area_cm2 < 3.0:
        return "sedang"
    else:
        return "besar"

def extract_physical_features(img, pixels_per_cm, pixels_per_mm):
    features = []
    contours, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        cnt = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(cnt)
        area_cm2 = cv2.contourArea(cnt) / (pixels_per_cm**2)
        perimeter_mm = cv2.arcLength(cnt, True) / pixels_per_mm
        length_mm = h / pixels_per_mm
        width_mm = w / pixels_per_mm
    else:
        area_cm2 = perimeter_mm = length_mm = width_mm = 0

    features.extend([area_cm2, perimeter_mm, length_mm, width_mm])

    area_mapping = {"kecil": [1,0,0], "sedang": [0,1,0], "besar": [0,0,1]}
    features.extend(area_mapping[categorize_area_cm2(area_cm2)])

    length_mapping = {"pendek": [1,0], "panjang": [0,1], "lainnya": [0,0]}
    features.extend(length_mapping[categorize_length_mm(length_mm)])

    glcm = graycomatrix(img, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
    features.extend([
        graycoprops(glcm, 'contrast')[0,0],
        graycoprops(glcm, 'dissimilarity')[0,0],
        graycoprops(glcm, 'homogeneity')[0,0],
        graycoprops(glcm, 'energy')[0,0],
        graycoprops(glcm, 'correlation')[0,0]
    ])
    return features

In [4]:
# =====================================================
# Preprocessing gambar & CapsuleNet utility
# =====================================================
def preprocess_image(img, target_size=(224,224)):
    img_resized = cv2.resize(img, target_size)
    img_resized = img_resized / 255.0
    return img_resized

def squash(vectors, axis=-1):
    s_squared_norm = K.sum(K.square(vectors), axis, keepdims=True)
    scale = s_squared_norm / (1 + s_squared_norm) / K.sqrt(s_squared_norm + K.epsilon())
    return scale * vectors

def create_capsule_layer(inputs, num_capsules=10, dim_capsule=16):
    u = Conv2D(filters=num_capsules*dim_capsule, kernel_size=3, strides=2, padding='valid', activation='relu')(inputs)
    u = Reshape(target_shape=[-1, dim_capsule])(u)
    u = Lambda(squash)(u)
    return u

In [5]:
# =====================================================
# Ekstraksi fitur MobileNet+CapsuleNet
# =====================================================
def extract_deep_features(images):
    base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224,224,3))
    for layer in base_model.layers:
        layer.trainable = False
    x = base_model.output
    capsule_features = create_capsule_layer(x)
    features_model = Model(inputs=base_model.input, outputs=capsule_features)
    deep_features = features_model.predict(images, verbose=0)
    deep_features = deep_features.reshape(deep_features.shape[0], -1)
    return deep_features

In [6]:
# =====================================================
# Gabungkan semua fitur
# =====================================================
def prepare_dataset(images_gray, labels, pixels_per_cm, pixels_per_mm):
    images_rgb = np.array([preprocess_image(cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)) for img in images_gray])
    deep_feats = extract_deep_features(images_rgb)
    phys_feats = np.array([extract_physical_features(img, pixels_per_cm, pixels_per_mm) for img in images_gray])
    X = np.hstack([deep_feats, phys_feats])
    y = np.array(labels)
    return X, y

In [7]:
# =====================================================
# Training & Evaluasi Random Forest
# =====================================================
def train_random_forest(X_train, y_train, n_estimators=100):
    rf = RandomForestClassifier(n_estimators=n_estimators, random_state=42)
    rf.fit(X_train, y_train)
    return rf

def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred, labels=np.unique(y_test)))
    print("Classification Report:\n", classification_report(y_test, y_pred, labels=np.unique(y_test)))

In [8]:
# =====================================================
# Fungsi Otsu & Otsu-Sinus
# =====================================================
def otsu_threshold(hist):
    sum_total = np.dot(np.arange(256), hist)
    best_t, max_sigma = 0, 0
    w0, sum0 = 0.0, 0.0
    for t in range(256):
        w0 += hist[t]
        sum0 += t * hist[t]
        if w0 == 0 or w0 == 1:
            continue
        w1 = 1 - w0
        mu0 = sum0 / w0
        mu1 = (sum_total - sum0) / w1
        sigma_b = w0 * w1 * (mu0 - mu1) ** 2
        if sigma_b > max_sigma:
            max_sigma, best_t = sigma_b, t
    return best_t

def otsu_sinus(hist):
    sum_total = np.dot(np.arange(256), hist)
    best_t, max_sigma = 0, 0
    w0, sum0 = 0.0, 0.0
    for t in range(256):
        w0 += hist[t]
        sum0 += t * hist[t]
        if w0 == 0 or w0 == 1:
            continue
        w1 = 1 - w0
        mu0 = sum0 / w0
        mu1 = (sum_total - sum0) / w1
        sigma_b = w0 * w1 * (math.sin(mu0 - mu1)) ** 2
        if sigma_b > max_sigma:
            max_sigma, best_t = sigma_b, t
    return best_t

def glcm_features(image):
    glcm = graycomatrix(image, distances=[1], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4],
                        symmetric=True, normed=True)
    return {
        "contrast": graycoprops(glcm, 'contrast').mean(),
        "correlation": graycoprops(glcm, 'correlation').mean(),
        "energy": graycoprops(glcm, 'energy').mean(),
        "homogeneity": graycoprops(glcm, 'homogeneity').mean()
    }

In [9]:
# =====================================================
# Main Program
# =====================================================
if __name__ == "__main__":
    dataset_path = "C:/Users/Naufal/Downloads/gabungan"
    pixels_per_cm = 50
    pixels_per_mm = pixels_per_cm / 10

    print("=== Load Dataset ===")
    images_gray, labels = load_dataset(dataset_path)

    print("Preparing features...")
    X, y = prepare_dataset(images_gray, labels, pixels_per_cm, pixels_per_mm)

    print("Split train/test...")
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    print("Training Random Forest...")
    rf_model = train_random_forest(X_train, y_train)

    print("Evaluating model...")
    evaluate_model(rf_model, X_test, y_test)

        # -------------------------------------------------
    # Demo Otsu & Otsu-Sinus untuk SEMUA gambar di folder
    # -------------------------------------------------
    folder_path = "C:/Users/Naufal/Downloads/gabungan"
    kernel = np.ones((3,3), np.uint8)

    for file_name in os.listdir(folder_path):
        if not file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue
        img_path = os.path.join(folder_path, file_name)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

        if img is None:
            print(f"❌ {file_name} tidak bisa dibaca")
            continue

        # === 1. Histogram ===
        hist = cv2.calcHist([img], [0], None, [256], [0, 256]).ravel()
        hist = hist / hist.sum()

        # === 2. Hitung threshold Otsu & Otsu-Sinus ===
        t_otsu = otsu_threshold(hist)
        t_sin = otsu_sinus(hist)
        print(f"{file_name} -> Otsu: {t_otsu}, Otsu-Sinus: {t_sin}")

        _, th_otsu = cv2.threshold(img, t_otsu, 255, cv2.THRESH_BINARY)
        _, th_sin = cv2.threshold(img, t_sin, 255, cv2.THRESH_BINARY)

        # === 3. Fitur GLCM ===
        features_sin = glcm_features(th_sin.astype(np.uint8))
        features_otsu = glcm_features(th_otsu.astype(np.uint8))
        print(f"📊 {file_name} | GLCM Otsu-Sinus: {features_sin}")
        print(f"📊 {file_name} | GLCM Otsu: {features_otsu}")

        # === 4. Gabung threshold dengan GLCM ===
        glcm_values = np.array(list(features_sin.values()))
        glcm_norm = (glcm_values - glcm_values.min()) / (glcm_values.max() - glcm_values.min() + 1e-6)
        glcm_weight = int((glcm_norm.mean() * 255))

        _, th_glcm = cv2.threshold(img, int((t_sin + glcm_weight)/2), 255, cv2.THRESH_BINARY)
        _, th_glcmotsu = cv2.threshold(img, int((t_otsu + glcm_weight)/2), 255, cv2.THRESH_BINARY)

        cleaned = cv2.morphologyEx(th_glcm, cv2.MORPH_OPEN, kernel, iterations=2)
        cleaned = cv2.morphologyEx(cleaned, cv2.MORPH_CLOSE, kernel, iterations=2)

        # === 5. Plot hasil ===
        titles = ['Original',
                  f'Otsu (T={t_otsu})',
                  f'Otsu-Sinus (T={t_sin})',
                  f'Otsu-Sinus+GLCM (T≈{int((t_sin+glcm_weight)/2)})',
                  f'Otsu+GLCM (T≈{int((t_otsu+glcm_weight)/2)})',
                  'Otsu-Sinus+GLCM+Morphology']
        images = [img, th_otsu, th_sin, th_glcm, th_glcmotsu, cleaned]

        plt.figure(figsize=(15, 4))
        for i in range(len(images)):
            plt.subplot(1, 6, i+1)
            plt.imshow(images[i], cmap='gray')
            plt.title(titles[i], fontsize=8)
            plt.axis('off')
        plt.suptitle(f"Hasil thresholding: {file_name}", fontsize=10)
        plt.tight_layout()
        plt.show()


=== Load Dataset ===
Preparing features...

Split train/test...
Training Random Forest...
Evaluating model...
Accuracy: 0.8340248962655602
Confusion Matrix:
 [[75  1  5]
 [ 3 66 11]
 [ 9 11 60]]
Classification Report:
               precision    recall  f1-score   support

      INPARI       0.86      0.93      0.89        81
        IR16       0.85      0.82      0.84        80
   ROJO LELE       0.79      0.75      0.77        80

    accuracy                           0.83       241
   macro avg       0.83      0.83      0.83       241
weighted avg       0.83      0.83      0.83       241

